In [1]:
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
import json
import datetime
import time
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [3]:
cases_and_deaths = {
    "date": "date",
    "dailycases": "newCasesByPublishDate",
    "dailydeaths": "newDeathsByDeathDate"
}

ventilators_v_admissions = {
    "date": "date",
    "hospitalcases": "hospitalCases",
    "ventilatorbeds": "covidOccupiedMVBeds"
}

age_distribution = {
    "malecases": "maleCases",
    "femalecases": "femaleCases"
}

england_only = [
    'areaType=nation',
    'areaName=England'
]

In [4]:
with open("timeseries.json", "rt") as INFILE:
    timeseriesdata = json.load(INFILE)

In [5]:
with open("ventilator.json","rt") as INFILE:
    ventdata=json.load(INFILE)

In [6]:
with open("agedistribution.json","rt") as INFILE:
    agedata=json.load(INFILE)

In [7]:
def parse_date(datestring):
    """Convert a date string into a pandas datetime object"""
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def min_age(agerange):
    agerange=agerange.replace('+','')
    start=agerange.split('_')[0]
    return int(start)

In [8]:
def timeseries_wrangle_data(timeseriesdata):
    timeseriesdatalist = timeseriesdata['data']
    timeseriesdates=[dictionary['date'] for dictionary in timeseriesdatalist]
    timeseriesdates.sort()
    startdate = parse_date(timeseriesdates[0])
    enddate = parse_date(timeseriesdates[-1])
    index = pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['dailycases','dailydeaths'])
    for entry in timeseriesdatalist:
        date=parse_date(entry['date'])
        for column in ['dailycases','dailydeaths']:
            if pd.isna(timeseriesdf.loc[date, column]):
                if entry[column]!= None:
                    value = float(entry[column])
                else:
                    value = 0.0
                timeseriesdf.loc[date, column]=value
    timeseriesdf.fillna(0.0, inplace=True)
    timeseriesdf['7daymovingavgcases'] = timeseriesdf['dailycases'].rolling(window=7).mean()
    timeseriesdf['7daymovingavgdeaths'] = timeseriesdf['dailydeaths'].rolling(window=7).mean()
    
    return timeseriesdf

In [9]:
def vent_wrangle_data(ventdata):
    ventdatalist = ventdata['data']
    ventdates = [dictionary['date'] for dictionary in ventdatalist]
    ventdates.sort()
    startdate = parse_date(ventdates[0])
    enddate = parse_date(ventdates[-1])
    index = pd.date_range(startdate, enddate, freq='D')
    ventdf = pd.DataFrame(index=index, columns=['hospitalcases','ventilatorbeds'])
    for entry in ventdatalist:
        date = parse_date(entry['date'])
        for column in ['hospitalcases','ventilatorbeds']:
            if pd.isna(ventdf.loc[date, column]):
                if entry[column]!= None:
                    value = float(entry[column])
                else:
                    value = 0.0
                ventdf.loc[date, column] = value
    ventdf.fillna(0.0, inplace=True)
    ventdf['noventilator']=(ventdf['hospitalcases'] - ventdf['ventilatorbeds'])
    ventdf.fillna(0.0, inplace=True)
    return ventdf

In [10]:
def age_wrangle_data(agedata):
    datadic=agedata['data'][0]
    males=datadic['malecases']
    females=datadic['femalecases']
    ageranges = [x['age'] for x in males]
    ageranges.sort(key=min_age)
    age_df = pd.DataFrame(index = ageranges, columns=['males', 'females'])
    for entry in males:
        ageband=entry['age']
        age_df.loc[ageband, 'males']=entry['value']
    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']
    
    return age_df

In [11]:
def access_api_timeseries():
    global timeseriesdata
    timeseriesdata = Cov19API(filters=england_only, structure=cases_and_deaths).get_json()

def access_api_testing():
    global testingdata
    testingdata = Cov19API(filters=england_only, structure=cases_and_deaths).get_json()
    
def access_api_vent():
    global ventdata
    ventdata = Cov19API(filters=england_only, structure=ventilators_v_admissions).get_json()

def access_api_age():
    global agedata
    agedata = Cov19API(filters=england_only, structure=age_distribution).get_json()

In [12]:
timeseriesapibutton = wdg.Button(
    description = 'Refresh Data',
    disabled=False,
    button_style='info',
    tooltip='Click to download Public Health England data',
    icon='refresh'
)

ventilatorapibutton = wdg.Button(
    description = 'Refresh Data',
    disabled=False,
    button_style='info',
    tooltip='Click to download Public Health England data',
    icon='refresh'
)

ageapibutton = wdg.Button(
    description = 'Refresh Data',
    disabled=False,
    button_style='info',
    tooltip='Click to download Public Health England data',
    icon='refresh'
)

In [13]:
timeseriescols=wdg.SelectMultiple(
    options=['dailycases','dailydeaths','7daymovingavgcases', '7daymovingavgdeaths'],
    value=['dailycases','dailydeaths','7daymovingavgcases', '7daymovingavgdeaths'],
    rows=4,
    description='Daily View',
    disabled=False
)

timeseriesscale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def timeseries_graph(graphcolumns, graphscale):
    graphcolumns = graphcolumns
    graphscale = graphscale
    if graphscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(graphcolumns)
    if ncols>0 and logscale==False:
        timeseriesdf.plot(title='Daily Cases Vs Deaths',
                         y=list(graphcolumns),
                         xlabel='Date',
                         ylabel='Count',
                         color={"dailycases": "orange", "dailydeaths": "red", "7daymovingavgcases": "green", "7daymovingavgdeaths": "blue"}).legend(loc='upper center')
    elif ncols>0 and logscale==True:
            timeseriesdf.plot(
                         logy=True,
                         title='Daily Cases Vs Deaths',
                         y=list(graphcolumns),
                         xlabel='Date',
                         ylabel='Count',
                         color={"dailycases": "orange", "dailydeaths": "red", "7daymovingavgcases": "green", "7daymovingavgdeaths": "blue"}).legend(loc='upper center')
    else:
        print("Click to select data for the chart")
        print("Ctrl + Click (Windows) / Command + Click (MacOS) to select multiple categories")

In [14]:
ventcols=wdg.SelectMultiple(
    options=['noventilator','ventilatorbeds'],
    value=['noventilator','ventilatorbeds'],
    rows=2,
    description='Hospitals:',
    disabled=False,
)

def ventilator_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        ventdf.plot(kind='area',
                   y=list(graphcolumns),
                   color={"noventilator": "yellow", "ventilatorbeds": "red"},
                   title="Proportion of hospital cases on ventilator",
                   xlabel='Date',
                   ylabel="Cases"
                   )
    else:
        print("Click to select data for the chart")
        print("Ctrl + Click (Windows) / Command + Click (MacOS) to select multiple categories")

In [15]:
agecols=wdg.SelectMultiple(
    options=['males', 'females'],
    value=['males', 'females'],
    rows=2,
    description='Sex:',
    disabled=False
)

def age_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        agedataframe.plot(kind='bar',
                         y=list(graphcolumns),
                         stacked=True,
                         color={"males": "red", "females": "pink"},
                          title="Age and Sex Case Distribution",
                          xlabel="Age",
                          ylabel="Cases"
                         )
    else:
        print("Click to select data for the chart")
        print("Ctrl + Click (Windows) / Command + Click (MacOS) to select multiple categories")

In [16]:
def refresh_ts_graph():
    current=timeseriesscale.value
    if current==timeseriesscale.options[0]:
        other=timeseriesscale.options[1]
    else:
        other=timeseriesscale.options[0]
    timeseriesscale.value=other
    timeseriesscale.value=current

In [17]:
def timeseries_callback(timeseriesapibutton):
    try:
        access_api_timeseries()
        global timeseriesdf
        timeseriesdf = timeseries_wrangle_data(timeseriesdata)
        timeseriesapibutton.icon='check'
        timeseriesapibutton.description='Data Refreshed'
        timeseriesapibutton.button_style='success'
        refresh_ts_graph()
        timeseriesapibutton.disabled=True
        time.sleep(5)
        timeseriesapibutton.description='Refresh Data'
        timeseriesapibutton.icon='refresh'
        timeseriesapibutton.button_style='info'
        timeseriesapibutton.disabled=False
    except:
        timeseriesapibutton.description='Error Occurred'
        timeseriesapibutton.button_style='warning'
        timeseriesapibutton.icon='fa-exclamation-triangle'
        timeseriesapibutton.disabled=True
        time.sleep(5)
        timeseriesapibutton.description='Refresh Data'
        timeseriesapibutton.icon='refresh'
        timeseriesapibutton.button_style='info'
        timeseriesapibutton.disabled=False

timeseriesapibutton.on_click(timeseries_callback)

In [18]:
def vent_callback(ventilatorapibutton):
    try:
        access_api_vent()
        global ventdf
        ventdf = vent_wrangle_data(ventdata)
        ventilatorapibutton.icon="check"
        ventilatorapibutton.description="Data Refreshed"
        ventilatorapibutton.button_style="success"
        ventilatorapibutton.disabled=True
        time.sleep(5)
        ventilatorapibutton.description='Refresh Data'
        ventilatorapibutton.icon='refresh'
        ventilatorapibutton.button_style='info'
        ventilatorapibutton.disabled=False
    except:
        ventilatorapibutton.description='Error Occurred'
        ventilatorapibutton.button_style='warning'
        ventilatorapibutton.icon='fa-exclamation-triangle'
        ventilatorapibutton.disabled=True
        time.sleep(5)
        ventilatorapibutton.description='Refresh Data'
        ventilatorapibutton.icon='refresh'
        ventilatorapibutton.button_style='info'
        ventilatorapibutton.disabled=False
        

ventilatorapibutton.on_click(vent_callback)

In [19]:
def age_callback(ageapibutton):
    try:
        access_api_age()
        global age_df
        age_df = age_wrangle_data(agedata)
        ageapibutton.icon="check"
        ageapibutton.description="Data Refreshed"
        ageapibutton.button_style="success"
        ageapibutton.disabled=True
        time.sleep(5)
        ageapibutton.description='Refresh Data'
        ageapibutton.icon='refresh'
        ageapibutton.button_style='info'
        ageapibutton.disabled=False
    except:
        ageapibutton.description='Error Occurred'
        ageapibutton.button_style='warning'
        ageapibutton.icon='fa-exclamation-triangle'
        ageapibutton.disabled=True
        time.sleep(5)
        ageapibutton.description='Refresh Data'
        ageapibutton.icon='refresh'
        ageapibutton.button_style='info'
        ageapibutton.disabled=False
        
ageapibutton.on_click(age_callback)

In [20]:
timeseriesdf = timeseries_wrangle_data(timeseriesdata)

# Covid-19 Dashboard
The following tiles use data from Public Health England to display key information on the Covid-19 pandemic in England.

## Daily Cases Vs Deaths
The below chart tracks the number of daily cases vs the number of daily deaths (defined as a death within 28 days of a positive covid-19 test) over time. The 7 day moving average of cases and deaths is also shown.

In [21]:
timeseriesoutput = wdg.interactive_output(timeseries_graph, {'graphcolumns': timeseriescols, 'graphscale': timeseriesscale})

controls = wdg.VBox([timeseriescols, timeseriesscale, timeseriesapibutton])
tsfinaldisplay = wdg.HBox([timeseriesoutput, controls])
display(tsfinaldisplay)

In [22]:
ventdf = vent_wrangle_data(ventdata)

## Mechanical Ventilator Cases as Proportion of all Hospital Admissions
The below chart tracks the number of patients in hospitals in England on a given day, where the admission is due to Covid-19. The number of patients on ventilators as a proportion of all hospital admissions can be observed. Patients requiring a mechanical ventilator are shown in red, with all other hospitalised Covid-19 patients shown in yellow.

In [23]:
ventoutput=wdg.interactive_output(ventilator_graph, {'graphcolumns': ventcols})

controls = wdg.VBox([ventcols, ventilatorapibutton])
finaldisplay = wdg.HBox([ventoutput, controls])
display(finaldisplay)

In [24]:
agedataframe = age_wrangle_data(agedata)

## Total Cases in England by Age and Sex Distribution
The below chart shows the distribution of confirmed Covid-19 positive test results across the population of England, by both age band and by sex. 

In [25]:
ageoutput=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

controls = wdg.VBox([agecols, ageapibutton])
finaldisplay = wdg.HBox([ageoutput, controls])
display(finaldisplay)